In [91]:
import pandas as pd 
import numpy as np 
from scipy.fft import fft, fftfreq
from scipy import signal
import matplotlib.pyplot as plt
import uuid
from sklearn.model_selection import train_test_split
from sklearn import svm 
from sklearn.linear_model import SGDClassifier
import sklearn
#from os import listdir
import os

%matplotlib inline


In [67]:
import sklearn

In [68]:
def clean1(filename): 
    data=pd.read_csv(filename,skiprows=4)
    data.columns=data.columns.str.strip()
    return data

def clean2(filename):
    data=pd.read_csv(filename, sep=",", header=None,skiprows=5).values
    
    
    return data

def clean3(filename):
    data=pd.read_csv(filename)
    
    data=data.drop("Epoch Time",axis=1)
    data=data.drop("Timestamp",axis=1)
    
    data=data.values
    return data

In [69]:
def get_data():
    arr = os.listdir('/Users/brysonpelechaty/Desktop/Capstone/Data')
    arr1=[]
    
    for file in os.listdir('/Users/brysonpelechaty/Desktop/Capstone/Data'):
        if file.endswith(".txt"):
            #print(os.path.join('/Users/brysonpelechaty/Desktop/Capstone/Data', file))
            arr1.append(os.path.join('/Users/brysonpelechaty/Desktop/Capstone/Data', file))
            
    for i in range(len(arr1)):
        data=clean1(arr1[i])
        
    data.head()
    
    return arr1
        #data=clean2(temp)
    

In [70]:
def search_string(filename):
    s1="ex01"
    s2="ex02"
    s3="ex03"
    s4="ex04"
    s5="ex05"
    s6="ex06"
    s7="ex07"
    s8="ex08"
    s9="ex09"
    
    check=[s1,s2,s3,s4,s5,s6,s7,s8,s9]
    
    
    fullstring = filename
    
    for i in range(len(check)):
        substring=check[i]
        if fullstring.find(substring)!= -1: 
            #print("Found")
            return substring

        #else:
            #print("Not found")

In [71]:
datalist=get_data()

In [72]:
#print(datalist)

In [73]:
fs = 1000
band = (15,50)

In [74]:
def notch(val, data, fs= 200):
    notch_freq_Hz = np.array([float(val)])
    for freq_Hz in np.nditer(notch_freq_Hz):
        bp_stop_Hz = freq_Hz + 3.0 * np.array([-1, 1])
        b, a = signal.butter(3, bp_stop_Hz / (fs / 2.0), 'bandstop')
        fin = data = signal.lfilter(b, a, data)
    return fin

In [75]:
def bandpass(start, stop, data, fs = 200):
    bp_Hz = np.array([start, stop])
    b, a = signal.butter(5, bp_Hz / (fs / 2.0), btype='bandpass')
    return signal.lfilter(b, a, data, axis=0)

In [76]:
def fft(data, fs):
    L = len(data)
    freq = np.linspace(0.0, 1.0 / (2.0 * fs **-1), L // 2)
    yi = np.fft.fft(data)[1:]
    y = yi[range(int(L / 2))]
    return freq, abs(y)

In [77]:
def fft_filter(data):
    
    channels = []
    
    bandpass_channels = []
    bandpass_notch_channels = []
    notch_channels = [] 
    fs = 1000
    band = (15,50)
    
    for i in range(8):
        channels.append(data[:,1+i].astype(np.float))
    t = len(channels[0])/fs
    time = np.linspace(0, t, len(channels[0])) 
   
    
    
    
    for i in range(len(channels)):
        notch_channels.append(notch(60,channels[i], fs = fs))
    
    #for i in range(len(notch_channels)):
        #plt.plot(time, notch_channels[i])
    
    
    for i in range(len(channels)):
        bandpass_channels.append(bandpass(band[0],band[1],channels[i], fs = fs))
    
    for i in range(len(notch_channels)):
        bandpass_notch_channels.append(bandpass(band[0],band[1],notch_channels[i], fs = fs)) 
        
        
    
    #for i in range(len(bandpass_notch_channels)):
     #   freq, y = fft(bandpass_notch_channels[i], fs)
      #  plt.plot(freq, y)
       # plt.ylabel("Magnintude of frequency")
       # plt.xlabel("Frequency in Hertz")
       # plt.title(title)
    #plt.ylim(0, 1e7)
    #plt.xlim(10,60) 
    
    
    return bandpass_notch_channels
    

In [78]:
def get_dataframes(data):
    bandpass_notch_channels=data
    df1=pd.DataFrame(columns=["Frequency","y"])
    df2=pd.DataFrame(columns=["Frequency","y"])
    df3=pd.DataFrame(columns=["Frequency","y"])
    df4=pd.DataFrame(columns=["Frequency","y"])
    df5=pd.DataFrame(columns=["Frequency","y"])
    df6=pd.DataFrame(columns=["Frequency","y"])
    df7=pd.DataFrame(columns=["Frequency","y"])
    df8=pd.DataFrame(columns=["Frequency","y"])
    for i in range(len(bandpass_notch_channels)):
        freq, y = fft(bandpass_notch_channels[i], fs)
        if(i==0):
            df1["Frequency"]=freq
            df1["y"]=y
        elif(i==1):
            df2["Frequency"]=freq
            df2["y"]=y
        elif(i==2):
            df3["Frequency"]=freq
            df3["y"]=y
        elif(i==3):
            df4["Frequency"]=freq
            df4["y"]=y
        elif(i==4):
            df5["Frequency"]=freq
            df5["y"]=y
        elif(i==5):
            df6["Frequency"]=freq
            df6["y"]=y
        elif(i==6):
            df7["Frequency"]=freq
            df7["y"]=y
        elif(i==7):
            df8["Frequency"]=freq
            df8["y"]=y
    
    return df1, df2, df3, df4, df5, df6, df7, df8

In [79]:
def make_vector(df1,df2,df3,df4,df5,df6,df7,df8): 
    
    vector=(get_freq_band(df1))
    sensor1=get_freq_band(df1)
    vector=vector.append(get_freq_band(df2))
    sensor2=get_freq_band(df2)
    vector=vector.append(get_freq_band(df3))
    vector=vector.append(get_freq_band(df4))
    vector=vector.append(get_freq_band(df5))
    vector=vector.append(get_freq_band(df6))
    vector=vector.append(get_freq_band(df7))
    vector=vector.append(get_freq_band(df8))
    
    
    #print(vector)
    #print("vector length is ",len(vector))
    
    sensorno=[]
    sensor="sensor"
    band=1
    counter=0
    
    for i in range(40):
        sensorno.append(sensor+str(band))
        counter+=1
        if(counter==5):
            band+=1
            counter=0
    vector["Sensor"]=sensorno
    
    
        
    
    values=[]
    
    for i in range(len(vector)):
        item=vector.iloc[i]["val"]
        values.append(item)
        
    #bar1=sensor1.plot.bar(x='band', y='val', legend=False)
    #bar2=sensor2.plot.bar(x="band", y="val",legend=False)
    
    vector.head()
    
    return vector

In [80]:
def get_freq_band(data): 
    fft_vals =data["y"]

# Get frequencies for amplitudes in Hz
    fft_freq = data["Frequency"]

# Define EEG bands
    eeg_bands = {'Delta': (0, 4),
                 'Theta': (4, 8),
                 'Alpha': (8, 12),
                 'Beta': (12, 30),
                 'Gamma': (30, 45)}

# Take the mean of the fft amplitude for each EEG band
    eeg_band_fft = dict()
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & 
                           (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = np.mean(fft_vals[freq_ix])

    # Plot the data (using pandas here cause it's easy)
    import pandas as pd
    df = pd.DataFrame(columns=['band', 'val'])
    df['band'] = eeg_bands.keys()
    df['val'] = [eeg_band_fft[band] for band in eeg_bands]
   # ax = df.plot.bar(x='band', y='val', legend=False)
   # ax.set_xlabel("EEG band")
   #ax.set_ylabel("Mean band Amplitude")
    #print(df)
    
    return df

In [81]:
def  do_experiment(data,expno):
    
    data=fft_filter(data)
    
    df1, df2, df3, df4, df5, df6, df7, df8=get_dataframes(data)
    
    data=make_vector(df1, df2, df3, df4, df5, df6, df7, df8)
    
    
    
    explist=[]
    for i in range(40):
        explist.append(expno) 
        
    data["Label"]=explist
    
    return data
    
    
    
    

In [82]:
def build_ML_Model(data):
    print("less go", data)
    brain_wave_dict={"Delta":0, "Theta":1,"Alpha":2,"Beta":3, "Gamma":4}
    
    label_dict={"ex01":0, "ex02":1,"ex03":2, "ex04":3,"ex05":4,"ex06":5,"ex07":6,"ex08":7,"ex09":8}
    
    
    data["band"]=data["band"].map(brain_wave_dict)
    data["Label"]=data["Label"].map(label_dict)
    y=data["Label"].values
   
    
    X=data
    X=X.drop("Sensor",axis=1)
    X=X.drop("Label",axis=1)
    X_encoded=pd.get_dummies(X,columns=["band"])
    
    yarr=[]
    for i in range(len(data["Label"])):
        yarr.append(data.iloc[i]["Label"])
        
    
    
    actual_y=pd.DataFrame(columns=["Label"])
    actual_y["Label"]=yarr
    
    print("here")
    print(X)
    print("here")
    print(actual_y)
    #X_train, X_test, y_train, y_test,= train_test_split(X_encoded,yarr, test_size=0.2)
    #lin_clf = svm.LinearSVC()
    #lin_clf.fit(X_train, y_train)
    
    #boobs=lin_clf.score(X_test,y_test)
    #print("the socre is",boobs)
    
    
    
    

In [83]:
def do_all(): 
    
    data_list=get_data()
    
    cleaned_data_list=[]
    
    main_data=pd.DataFrame
    temp_data=pd.DataFrame
    
    for i in range(len(data_list)):
        temp_data=do_experiment(clean2(data_list[i]),search_string(datalist[i])) 
        
        cleaned_data_list.append(temp_data) 
        
    
    temp_data=do_experiment(clean3("ex03-2022-07-19-2252.csv"),"ex03")
    cleaned_data_list.append(temp_data)
    temp_data=do_experiment(clean3("ex02-2022-07-18-2.csv"),"ex02")
    cleaned_data_list.append(temp_data)
    temp_data=do_experiment(clean3("ex04-2022-07-1.csv"),"ex04")
    cleaned_data_list.append(temp_data)
    temp_data=do_experiment(clean3("ex05-2022-07-18-22.csv"),"ex05")
    cleaned_data_list.append(temp_data)
    temp_data=do_experiment(clean3("ex06-2022-07-19-2.csv"),"ex06")
    cleaned_data_list.append(temp_data)
    temp_data=do_experiment(clean3("ex07-2022-07-18-22.csv"),"ex07")
    cleaned_data_list.append(temp_data)
    temp_data=do_experiment(clean3("ex07-2022-07-19-22.csv"),"ex07")
    cleaned_data_list.append(temp_data)
    temp_data=do_experiment(clean3("ex08-2022-07-18-2.csv"),"ex08")
    cleaned_data_list.append(temp_data)
    
    print(len(cleaned_data_list))
    
    main_data=pd.concat(cleaned_data_list)
    
    #build_ML_Model(main_data)
        
    return main_data
        
        
        
    


In [84]:
def pre_process(data):
    
    temp_data=do_experiment(clean2(data_list[0]),search_string(datalist[0]))
    brain_wave_dict={"Delta":0, "Theta":1,"Alpha":2,"Beta":3, "Gamma":4}
    
    label_dict={"ex01":0, "ex02":1,"ex03":2, "ex04":3,"ex05":4,"ex06":5,"ex07":6,"ex08":7,"ex09":8}
    
    data["band"]=data["band"].map(brain_wave_dict)
    data["Label"]=data["Label"].map(label_dict)
    y=data["Label"].values
    
    X=data
    X=X.drop("Sensor",axis=1)
    X=X.drop("Label",axis=1)
    X_encoded=pd.get_dummies(X,columns=["band"])
    
    
    return x,y

In [85]:
def do_all2(): 
    data_list=get_data()
    classify=[0,1,2,3,4,5,6,7,8]
    
    cleaned_data_list=[]
    clf = sklearn.linear_model.SGDClassifier()
    main_data=pd.DataFrame
    temp_data=pd.DataFrame
    #print(len(data_list))
    #print(clf)
    temp_data=do_experiment(clean2(data_list[0]),search_string(data_list[0]))
    brain_wave_dict={"Delta":0, "Theta":1,"Alpha":2,"Beta":3, "Gamma":4}
    
    label_dict={"ex01":0, "ex02":1,"ex03":2, "ex04":3,"ex05":4,"ex06":5,"ex07":6,"ex08":7,"ex09":8}
    
    temp_data["band"]=temp_data["band"].map(brain_wave_dict)
    temp_data["Label"]=temp_data["Label"].map(label_dict)
    y=temp_data["Label"].values
    
    X=temp_data
    X=X.drop("Sensor",axis=1)
    X=X.drop("Label",axis=1)
    X_encoded=pd.get_dummies(X,columns=["band"])
    print(X_encoded)
    print(y)
    
    #clf.partial_fit(X_encoded, y,classes=classify)
    
    main_data=do_experiment(clean2(data_list[1]),search_string(data_list[1]))
    
    #x_new,y_new=pre_process(temp_data)
    #clf.partial_fit(x_new,y_new)
    print("here")
          
    print(main_data.head())
    
    
    
    
        
    
    
    
    

In [93]:
do_all2()




ValueError: too many values to unpack (expected 2)

In [ ]:
#build_ML_Model(data)

In [ ]:
data1=pd.read_csv("ex03-2022-07-19-2252.csv")

In [ ]:
data1.head()